In [1]:
import pandas as pd

In [5]:

# Load the dataset
file_path = "processed_data\hourly_event_attendance.csv"  # Update this if needed
df = pd.read_csv(file_path)

# Convert the 'Hour' column to a uniform datetime format (keeping only hour-level precision)
df['Hour'] = pd.to_datetime(df['Hour']).dt.floor('H')

# Remove duplicate rows
df_cleaned = df.drop_duplicates()

# Save the cleaned dataset to a local directory
cleaned_file_path = "cleaned_hourly_event_attendance.csv"  # Change the path as needed
df_cleaned.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved at: {cleaned_file_path}")


<>:2: SyntaxWarning: invalid escape sequence '\h'
<>:2: SyntaxWarning: invalid escape sequence '\h'
C:\Users\Soumya Pandey\AppData\Local\Temp\ipykernel_39336\404873728.py:2: SyntaxWarning: invalid escape sequence '\h'
  file_path = "processed_data\hourly_event_attendance.csv"  # Update this if needed
C:\Users\Soumya Pandey\AppData\Local\Temp\ipykernel_39336\404873728.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Hour'] = pd.to_datetime(df['Hour']).dt.floor('H')


Cleaned dataset saved at: cleaned_hourly_event_attendance.csv


In [7]:
file_path = "weather_energy.csv"
df2 = pd.read_csv(file_path)

# Display the first few rows to understand the structure
df2.head()

# Convert datetime column to pandas datetime format
df2['datetime'] = pd.to_datetime(df2['datetime'])

# Set datetime as index
df2.set_index('datetime', inplace=True)

# Aggregate data to hourly intervals by taking the mean
df2_hourly = df2.resample('H').mean()

# Reset index to make datetime a column again
df2_hourly.reset_index(inplace=True)


C:\Users\Soumya Pandey\AppData\Local\Temp\ipykernel_39336\2110915090.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2_hourly = df2.resample('H').mean()


In [10]:
# Display the first few rows of the aggregated data
df2_hourly.head()


,datetime,Unnamed: 0,Power demand,temp,dwpt,rhum,wdir,wspd,pres,year,month,day,hour,minute,moving_avg_3
0,2021-01-01 00:00:00,2.5,1975.541667,8.0,6.90,93.0,0.0,0.0,1017.00,2021.0,1.0,1.0,0.0,42.5,1976.777500
1,2021-01-01 01:00:00,11.5,1805.590000,8.0,6.90,93.0,0.0,0.0,1017.00,2021.0,1.0,1.0,1.0,27.5,1822.649167
2,2021-01-01 02:00:00,23.5,1665.914167,7.5,6.25,92.0,0.0,0.0,1018.05,2021.0,1.0,1.0,2.0,27.5,1675.064722
3,2021-01-01 03:00:00,35.5,1607.331667,7.0,6.30,95.5,0.0,0.0,1018.05,2021.0,1.0,1.0,3.0,27.5,1608.205278
4,2021-01-01 04:00:00,47.5,1676.528333,6.5,6.50,100.0,0.0,0.0,1017.00,2021.0,1.0,1.0,4.0,27.5,1664.828333


In [11]:
cleaned_file_path2 = "aggregated_weather.csv"  # Change the path as needed
df2_hourly.to_csv(cleaned_file_path2, index=False)

print(f"Cleaned dataset saved at: {cleaned_file_path2}")

Cleaned dataset saved at: aggregated_weather.csv


In [18]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

# Load datasets
weather_df = pd.read_csv('aggregated_weather.csv')
hourly_df = pd.read_csv('cleaned_hourly_event_attendance.csv')

# Convert datetime columns to pandas datetime format
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
hourly_df['datetime'] = pd.to_datetime(hourly_df['Hour'])

# Resample weather data to hourly by averaging every 15-minute intervals
weather_hourly = weather_df.resample('H', on='datetime').mean().reset_index()

# Merge both datasets on 'datetime'
merged_df = pd.merge(hourly_df, weather_hourly, on='datetime', how='inner')

# Check for NaNs and Infinite values
merged_df = merged_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Power demand', 'Estimated_Attendance'])

# Compute overall Pearson correlation
if len(merged_df) > 1:
    overall_corr, _ = pearsonr(merged_df['Power demand'], merged_df['Estimated_Attendance'])
    print(f"Overall Pearson correlation: {overall_corr:.4f}")
else:
    print("Not enough data after cleaning to compute correlation.")

# Group by event_type and compute correlation for each
event_correlations = {}
for event, group in merged_df.groupby('EventCategory'):
    if len(group) > 1:  # Ensure enough data points for correlation
        try:
            event_corr, _ = pearsonr(group['Power demand'], group['Estimated_Attendance'])
            event_correlations[event] = event_corr
        except ValueError:
            continue  # Skip event types with insufficient valid data

# Sort events by absolute correlation (strongest first)
sorted_events = sorted(event_correlations.items(), key=lambda x: abs(x[1]), reverse=True)

# Print the sorted event categories and their correlation
print("\nEvent Type Correlations (sorted from max to min absolute correlation):")
for event, correlation in sorted_events:
    print(f"{event}: {correlation:.4f}")

# Find the event with the highest correlation
if sorted_events:
    most_correlated_event, highest_corr = sorted_events[0]
    print(f"\nEvent with highest correlation: {most_correlated_event} ({highest_corr:.4f})")
else:
    print("\nNo valid event correlations found.")


Overall Pearson correlation: -0.0052

Event Type Correlations (sorted from max to min absolute correlation):
Protest & Demonstration: -0.0474
Religious Event: -0.0244
Public Gathering: -0.0130
Sports & Competition: -0.0054
Cultural & Tourism Event: -0.0052
Other: -0.0010

Event with highest correlation: Protest & Demonstration (-0.0474)


C:\Users\Soumya Pandey\AppData\Local\Temp\ipykernel_39336\2278790158.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  weather_hourly = weather_df.resample('H', on='datetime').mean().reset_index()
